---
**<h1><center> </center></h1>**

**<h1><center> Proyecto: Clasificación de cáncer de piel </center></h1>**

**<h2><center> [tipos] </center></h2>**

**<h2><center> Visión Por Computador - 2024-2</center></h2>**

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c4/Escudo_unal_2016.png" width="100"></center>

---


<h1> Integrantes: </h1>

* Camilo Loaiza Fonnegra: cloaiza@unal.edu.co

* Luis Santiago Maya Restrepo: lmayar@unal.edu.co

* Jhon Sebastián Sepúlveda Nomesque: jsepulvedan@unal.edu.co

* Luis Alexander Torres Aguirre: luatorresag@unal.edu.co

* Andrés Felipe Zapata Osorio: anzapatao@unal.edu.co


# Descripción del Trabajo

#Descripción del Dataset

## Visualización de las Imágenes

## Análisis de Ruido en las Imágenes

## Análisis de Modelo de Color

## Histogramas